In [32]:
import cogent3
from cogent3 import load_unaligned_seqs
from cogent3 import load_aligned_seqs
from cogent3 import get_app

In [40]:
folder = '/xdisk/masel/uliseshmc/EstimatingUd/primates10_112/aligns_ortho/'
sequence = 'ENSG00000183831.fa'
aln = load_unaligned_seqs(folder + sequence, moltype='dna')
aln = aln.rename_seqs(lambda x: x.split("-")[0])
aln

,0
pan_troglodytes,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA
gorilla_gorilla,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA
nomascus_leucogenys,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAAGAAGA
chlorocebus_sabaeus,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAAATTTTCTCACAGCAAGAAGA
pan_paniscus,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA
microcebus_murinus,AATGGAGTCAGAAGGACCCCCAGAGTCAGTGAGTTCAGAAAAATCAGTATTTTTCTCACA
homo_sapiens,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA
pongo_abelii,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA
macaca_fascicularis,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA
macaca_mulatta,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA


In [41]:
hcm = aln.take_seqs(["homo_sapiens", "pan_troglodytes", "gorilla_gorilla"]).degap()
name_map = {
    "homo_sapiens": "Human",
    "pan_troglodytes": "Chimpanzee",
    "gorilla_gorilla": "Gorilla"
}
def renamer(name):
    return name_map.get(name, name)
hcm = hcm.rename_seqs(renamer)
hcm

,0
Human,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA
Chimpanzee,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA
Gorilla,AATGGAGTCAGAAGGACCTCCAGAGTCAGAGAGTTCAGAATTTTTCTCACAGCAGGAAGA


In [42]:
homoseq = hcm.get_seq("Human")
translated = homoseq.get_translation(include_stop=True)
for i in range(0, len(translated)):
    if translated[i] == "*":
        print(i)

22
55
58
63
90
93
104
134
151
156
159
183
185
193
208
219
220
249
255
262


In [44]:
trimmed_hcm = trim_stops(hcm)

homoseq = trimmed_hcm.get_seq("Human")
translated = homoseq.get_translation(include_stop=True)
for i in range(0, len(translated)):
    if translated[i] == "*":
        print(i)

22
55
58
63
90
93
104
134
151
156
159
183
185
193
208
219
220
249
255
262


In [45]:
#Doing nucleotide model meanwhile. I got to change this for codon eventually
trim_stops = get_app("trim_stop_codons")

#Check the tree! 
tree = "(Human:0.06,Chimpanzee:0.06,Gorilla:0.1)"
codon_aligner = get_app("progressive_align", "codon", guide_tree=tree)

align_app = trim_stops + codon_aligner

aln_codonaligned = align_app(hcm)
aln_codonaligned

NotCompleted(type=ERROR, origin=progressive_align, source="ENSG00000183831.fa", message="'TGA' at 'Human':66 not in alphabet")

In [38]:
ffold = get_app("take_codon_positions", fourfold_degenerate=True)
#Do I need omit_gap_pos() or degap()
aln_ffold = ffold(aln_codonaligned).omit_gap_pos()

In [39]:
sm = get_app("model", "GN", time_het="max", optimise_motif_probs=True, show_progress=True)
result = sm(aln_ffold)
lengthstree = result.lf.get_lengths_as_ens()
lengthstree

   0%|          |00:00<?

   0%|          |00:00<?

{'Human': np.float64(9.99997582679352e-07),
 'Chimpanzee': np.float64(9.999981644403043e-07),
 'Gorilla': np.float64(1.0000012313770403e-06)}